# Module 4 : RNN et attention pour le texte 


Vincent Guigue, inspiré de
Nicolas Baskiotis (nicolas.baskiotis@isir.upmc.fr) Benjamin Piwowarski (benjamin@piwowarski.fr)  -- MLIA/LIP6, ISIR, Sorbonne Université

La représentation moyenne étudiée au module précédent trouve vite ses limites lorsque l'on s'attaque à des tâches plus complexes (traduction, résumé automatique, ...) où ce qui nous intéresse est à une fine granularité dans le texte. Ce module explore les architectures récurrentes qui permettent de bien mieux traiter le texte. 

Nous allons étudier la *génération libre de texte* en utilisant `IMDB`.

La génération de séquence consiste  à produire  une séquence de symboles discrets à partir d'une séquence en entrée. L'objectif est donc de décoder à partir de l'état caché une distribution  de probabilités multinomiale sur les symboles à engendrer. Il faut donc que la  dimension de sortie du RNN (du décodeur) soit égale au nombre de symboles considérés. Il faut par ailleurs utiliser un *softmax* pour obtenir une distribution à partir du décodage de l'état caché. Le coût cross-entropie est adapté pour apprendre cette distribution.

Une fois le réseau appris, la génération se fait (soit à partir d'un début de séquence, soit à partir d'un état initial vide) en choisissant le symbole le plus probable dans la distribution multinomiale décodée à chaque pas de temps. Ce symbole est ensuite considéré comme entrée au pas de temps suivant et la génération se poursuit itérativement. Une autre possibilité est d'échantillonner suivant la multinomiale pour obtenir plusieurs échantillons. La génération se poursuit jusqu'à produire le token *\<eos\>*.

La supervision se fait à chaque pas de temps (contrairement à la classification où la supervision se fait au dernier pas de temps) : c'est un exemple de réseau many-to-many. On veut inciter le réseau à produire à l'instant $t+1$ le token suivant de notre séquence. Ainsi à tout pas de temps $t$ on connaît la supervision. Le gradient du coût à propager est la somme de tous les coûts à chaque instant. Il ne faut pas oublier cependant à ne pas prendre en compte les tokens de padding dans le calcul du coût.

In [ ]:
from packaging import version
from pathlib import Path
from itertools import chain
from typing import Iterable, List, Tuple

import shutil
import torchtext
assert version.Version(torchtext.__version__) >= version.Version("0.9.0")
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab, FastText
from collections import Counter
from tqdm.autonotebook import tqdm
from torch.nn.utils.rnn import pad_sequence
import sentencepiece as spm
from IPython.display import display, HTML
import os


cachepath = os.path.expanduser('~/.local/data')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BASEPATH = Path("xp/generation")
TB_PATH =  BASEPATH / "logs"
TB_PATH.mkdir(parents=True, exist_ok=True)

display(HTML("<h2>Informations</h2><div>Pour visualiser les logs, tapez la commande : </div>"))
print(f"tensorboard --logdir {Path(TB_PATH).absolute()}")

## Chargement et pré-traitement des données (XSum)

Ci-dessous les fonctions qui permettent de charger les données et de les préparer (en lots/batchs)

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum", split={"train": "train[:10%]", "validation": "validation[:5%]", "test": "validation[5%:]"})
metric = load_metric("rouge")

# Affiche les champs 
raw_datasets


# Extrait du jeu de données

Voici un extrait aléatoire du jeu de données, avec un document (à résumé) et le résumé attendu

In [ ]:
print(raw_datasets["train"][0])

## Réduction du vocabulaire

Les réseaux récurrents sont lourds à mettre en oeuvre : pour chaque mini-batch il faut itérer sur la longueur des séquences. Il est donc judicieux de réduire autant que possible les dimensionalités à traiter.
 
Le pré-traitement des textes repose sur une étape de segmentation où le texte est découpé en unités linguistiques. Pendant longtemps le niveau choisi était le mot (= chaîne alphanumérique entourée d'espace); depuis quelques années, des alternatives ont été (ré)explorées avec les nouveaux modèles neuronaux. 

Une des segmentations les plus efficaces à l'heure actuelle est le découpage en n-grammes variables (**subword units**) popularisé par le Byte-Pair Encoding (BPE) en 2016. Ces segmentations ont l'avantage d'avoir un vocabulaire de taille fixe qui couvre au mieux le jeu de données, et permet d'éviter le problème des mots inconnus.

Par exemple, *You shoulda got David Carr of Third Day to do it* sera segmenté en ```"\_You", "\_should", "a", "\_got", "\_D", "av", "id", "\_C", "ar", "r", "\_of", "\_Th", "ir", "d", "\_Day", "\_to", "\_do", "\_it"```

où les séquences fréquentes (ex. You, should) sont extraites directement alors que des séquences moins fréquentes (ex. David, Carr) sont segmentées en plusieurs parties.

La librairie [sentencepiece](https://github.com/google/sentencepiece/blob/master/python/README.md)</a> permet une telle segmentation. Les tokens Unknown *\<unk\>*, *BOS* (begin of sequence, *\<s\>*) and *EOS* (end of sequence, *\</s\>*) sont prédéfinis, mais vous pouvez en ajouter d'autres avec **user_defined_symbols**.
    
Nous allons également considéré qu'un sous-échantillon des exemples pour rendre plus rapide l'apprentissage.

Ne pas oublier de préfixer chaque phrase par le token *\<bos\>* et le suffixé par *\<eos\>*, cela nous servira dans des tâches futures.

La suite est classique : remarquez dans **pad_sequence** que cette fois on ne passe pas l'argument **batch_first=True** pour garder les conventions usuelles RNNs : la première dimension du tenseur est la longueur, la deuxième la taille du batch, la troisième la dimension d'embedding.

In [ ]:
class Preprocessor:
    def process_batch(self, batch: List[str]):            
        return pad_sequence([self.string2idx(text) for text in batch])

class SentencePiecePreprocessor(Preprocessor):
    """Tokenizer Sentence Piece"""
    def embeddings(self, dimension: int):
        return nn.Embedding(self.vocab_size, dimension, padding_idx=self.pad_index)
    
    @property
    def vocab_size(self):
        return self.spm_tokenizer.vocab_size()

    def __init__(self, name: str, sentences: Iterable[str], vocab_size=1000, force=False):
        modelpath = BASEPATH / f"{name}-{vocab_size}.model"
        if not modelpath.exists() or force:
            print(f"Entraînement de SPM, sortie {BASEPATH}/{name}-{vocab_size}", flush=True)
            spm.SentencePieceTrainer.train(
                sentence_iterator=iter(sentences), 
                model_prefix=f"{BASEPATH}/{name}-{vocab_size}", 
                vocab_size=vocab_size,
                pad_id=0,                
                unk_id=1,
                bos_id=2,
                eos_id=3
            )
            
        self.spm_tokenizer = spm.SentencePieceProcessor()
        self.spm_tokenizer.load(str(modelpath))

        self.pad_index = self.spm_tokenizer.pad_id()
        self.eos_index = self.spm_tokenizer.eos_id()
        self.bos_index = self.spm_tokenizer.bos_id()
        self.oov_index = None

        self.spm_tokenizer.SetEncodeExtraOptions("bos:eos")

    def string2idx(self, s: str) -> torch.Tensor:
        return torch.tensor(self.spm_tokenizer.EncodeAsIds(s))

    def tokenizer(self, x):
        """Segmentation du texte en sous-mots"""
        return self.spm_tokenizer.encode_as_pieces(x)

    def id2token(self, ix):
        return self.spm_tokenizer.IdToPiece(ix)

    def decode(self, ids: List[int]):
        return self.spm_tokenizer.Decode(ids)


# Si vous voulez essayer avec des mots
# preprocessor = FastTextProcessor()

preprocessor = SentencePiecePreprocessor("spm", (row["summary"] for row in raw_datasets["train"]), force=False)
print(f"Vocabulary size: {preprocessor.vocab_size}")

On essaie notre nouveau tokenizer sur "hello world" pour voir ce qui se passe. Notez :

1. Le découpage en sous-mots
1. La présence de tokens spéciaux (début `<s>` et fin de séquence `</s>`)


In [ ]:
preprocessor.tokenizer("hello world")

et maintenant, on traite notre jeu de données en entier avec la fonction `map` de datasets et une fonction de pré-traitement `preprocess_fn`

## <span class="alert-success"> Exercice : RNN simple </span>

Le RNN utilisé pour la génération est composé :
* d'une [couche d'embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) pour la représentation des tokens
* d'une <a href=https://pytorch.org/docs/stable/generated/torch.nn.RNN.html> cellule RNN</a>
* et d'un *décodeur*, chargé de décoder l'état latent du RNN vers la classe *positive* ou *négative*, sous la forme d'un réseau linéaire.

Le **forward** d'un module **RNN** retourne deux tenseurs, le premier correspond au tenseur **output** de  taille `Longueur x Batch x Taille du vocabulaire`, le deuxième au tenseur **hidden** qui correspond juste au dernier état caché. Ce tenseur du dernier état caché est très utile lorsque l'on veut poursuivre l'inférence à partir de l'état de sortie pour initialiser l'état caché du réseau (cf fonction `generate`).

Dans la cellule suivante, la boucle d'apprentissage (teacher forcing) est donnée. Vous pouvez modifier le code, si vous le souhaitez, pour utiliser des techniques d'apprentissage plus évoluées (utilisation de la sortie du RNN plutôt que de la vérité de terrain).

In [ ]:
TRAIN_BATCHSIZE = 128
TEST_BATCHSIZE = 512

def train_1to1(preprocessor, model, epochs, raw_datasets):
    """Fonction d'entraînement (teacher forcing)"""
    print(f"Training {model.name}")
    
    # On nettoie le rep. de log
    shutil.rmtree(f"{TB_PATH}/{model.name}", ignore_errors = True)
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    
    optim = torch.optim.Adam(model.parameters(), lr=1e-3)
    model = model.to(device)

    train_loader = DataLoader(raw_datasets["train"], TRAIN_BATCHSIZE, shuffle=True)
    test_loader = DataLoader(raw_datasets["test"], TEST_BATCHSIZE, shuffle=False)
    loss = nn.CrossEntropyLoss(ignore_index=preprocessor.pad_index)
    
    for epoch in tqdm(range(epochs)):
        cumloss, count =  0, 0
        model.train()
        for batch in train_loader:
            optim.zero_grad()
            x = preprocessor.process_batch(batch["summary"]).to(device)
        
            # Mode "Teacher forcing"
            x, y = x[:-1], x[1:]
            yhat, hidden = model.forward(x)
            l = loss(yhat.view(-1,yhat.size(2)), y.view(-1))
            l.backward()
            optim.step()
            cumloss += l*len(x)
            count += len(x)
        writer.add_scalar('loss/train', cumloss/count, epoch)

        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, count = 0, 0
                for x in test_loader:
                    x = preprocessor.process_batch(batch["summary"])
                    x, y = x[:-1].to(device), x[1:].to(device)
                    yhat, hidden = model(x)
                    cumloss += loss(yhat.view(-1,yhat.size(2)),y.view(-1))*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)


#  Donnez l'implémentation du module RNN

Il ne vous reste plus qu'à définir l'initialisation des modules (`__init__`) ainsi que leur utilisation (`forward`)

In [ ]:

class RNN(nn.Module):
    def __init__(self, embeddings, hidden_dim):
        super().__init__()
        self.name = f"rnn-{embeddings.embedding_dim}-{hidden_dim}"
        self.embeddings = embeddings

        # definir la partie recurrente dim_embedding => dim_h
        # prediction de char: le predicteur doit revenir dans l'espace des embeddings
        #  TODO 
        
    
    def forward(self, x, h_0=None):
        #  TODO 
        # Attention de bien prendre en compte h_0 dans l'appel du RRN
        

# On récupère des embeddings pour le modèle (dimension 50)
embeddings = preprocessor.embeddings(50)

# Création du RNN avec 100 états cachés
rnn_model = RNN(embeddings, 100)


In [ ]:

# Entraînement du modèle: 37 minutes chez moi => modèle chargeable dans la case suivante
train_1to1(preprocessor, rnn_model, 50, raw_datasets)

In [ ]:
import os

def save_model(preprocessor, model,fichier): # pas de sauvegarde de l'optimiseur ici
      """ sauvegarde du modèle dans fichier """
      state = {'model_state': model.state_dict(), 'preprocessor': preprocessor}
      torch.save(state,fichier) # pas besoin de passer par pickle
 
def load_model_RNN(fichier):
      """ Si le fichier existe, on charge le modèle  """
      if os.path.isfile(fichier):
          state = torch.load(fichier)
          preprocessor = state['preprocessor']
          embeddings = preprocessor.embeddings(50)
          model = RNN(embeddings, 100)
          model.load_state_dict(state['model_state'])
          return preprocessor, model

In [ ]:
# sauvegarde
# save_model(preprocessor, rnn_model, "model/rnn-gen")

# chargement ATTENTION à ne pas écraser un modèle appris
preprocessor,rnn_model  = load_model_RNN("model/rnn-gen")


Afin de visualiser ce que génère le modèle, nous pouvons utiliser une méthode simple qui consiste à échantilloner à chaque étape le mot à générer, puis à conditionner en fonction de ce qui a été choisi :

In [ ]:
def generate(preprocessor: Preprocessor, model, start="it is", maxlength=50):
    """Génère une suite de tokens en utilisant la distribution de probabilité du modèle"""

    assert start, "le début de la phrase doit être indiqué"

    with torch.no_grad():
        x = preprocessor.process_batch([start]).to(device)
        generated = []
        y_t, s_t = model(x)
        # for borné sur maxlength
        #   tirage d'un mot selon les y
        #   si le mot n'est pas eos
        #       reappeler le modèle (il faut un .unsqueeze(0) sur le mot tiré)
        
        #  TODO 
        return start + " " + preprocessor.decode(generated)

generate(preprocessor, rnn_model, start="i am curious: yellow")


## <span class="alert-success"> Exercice : GRU et LSTM </span>

Utilisez à la place de la cellule RNN usuelle un GRU (ou un LSTM, les GRUs sont un peu plus faciles à utiliser) et comparez les résultats.

In [ ]:
# le code est identique... 
#  cellule nn.RNN => nn.LSTM

#  TODO 


In [ ]:

# Entraînement du modèle
train_1to1(preprocessor, lstm_model, 50, raw_datasets)


In [ ]:
def load_model_LSTM(fichier):
      """ Si le fichier existe, on charge le modèle  """
      if os.path.isfile(fichier):
          state = torch.load(fichier)
          preprocessor = state['preprocessor']
          embeddings = preprocessor.embeddings(50)
          model = LSTM(embeddings, 100)
          model.load_state_dict(state['model_state'])
          return preprocessor, model

In [ ]:
# sauvegarde
# save_model(preprocessor, lstm_model, "model/lstm-gen")

# chargement (attention à ne pas écraser !!!)
preprocessor,lstm_model  = load_model_LSTM("model/lstm-gen")


In [ ]:
generate(preprocessor, lstm_model, start='''"i am curious: yellow" is a risible and pretentious''')


# Améliorer la génération de texte


Lors de vos tentatives de génération, vous observerez que en prenant à chaque pas de temps l'argmax vous obtiendrez très peu souvent des phrases intelligibles (vous faîtes en fait une approximation gloutonne du maximum de vraisemblance). Même l'échantillonage dans la distribution inférée ne rend pas bien meilleur le résultat. La solution usuelle consiste à utiliser un *beam search* pour approximer l'argmax sur toute la séquence engendrée : le beam search consiste à conserver à tout moment $t$ un ensemble de $K$ séquences (et leur log-probabilité associée); à l'étape $t+1$, on génère pour chacune des  séquences  $s$ les $K$ symboles les plus probables étant donnée $s$. Puis on sélectionne les $K$ séquence de taille $t+1$ les plus probables (et on ré-itère).

La qualité des séquences générées peut encore être améliorée en utilisant des techniques d'échantillonage telle que le Nucleus Sampling qui consiste à définir la probabilité de génération en ne considérant que les caractères les plus probables, en définissant un seuil $\alpha$ (un hyperparamètre, ex. 0.95) qui permet de sélectionner seulement les sorties permettant de couvrir au mieux cette masse de probabilité. Formellement, si $I_\alpha(p, s)$ est le plus petit ensemble de symboles tel que 
$$ \sum_{I_\alpha(p, s)} {p(y|s)}  \ge \alpha $$

La probabilité **nucleus** est définie comme 
$$
p_{\textrm{nucleus } K}(y|s_t) =
  1_{y \in I_\alpha(p, s_t)} \times \frac{p(y|s_t)}{\sum_{y^\prime\in I_\alpha(p, s_t)} p(y^\prime|s_t)}
$$

Implémentez la génération beam-search avec nucleus.

In [ ]:
# representation de la chaine de départ

def compute_h0(preprocessor, model, start):
    with torch.no_grad():
        res = ""
        if start:
            tids = preprocessor.string2idx(start)#[:-1]  # processing de la chaine start
            x = torch.LongTensor(tids).to(device)       # construction du tenseur d'indice
            output, hidden = model.forward(x.unsqueeze(1))  # traitement de la chaine
            res = tids
            # print(res)
        else:
            state = None
    return [int(x) for x in res], output[-1], hidden    # recuperation de l'ensemble des états cachés


In [ ]:
res, output, state =  compute_h0(preprocessor, rnn_model, "i was happy")
print(preprocessor.decode(res))


In [ ]:

def generate_beam(preprocessor, model, k, start="", maxlen=50):
    with torch.no_grad():
        res, output, state = compute_h0(preprocessor, model, start)
        
        # état, chaîne, log prob, length, eos
        candidates = [ (output, state, res, 0, len(preprocessor.string2idx(start)), False) ]
        
        
        for i in range(maxlen):
            new_candidates = []
            for output, state, res, logp, length, is_eos in candidates:
                if is_eos:
                    new_candidates.append((None, None, res, logp, length, None))
                else:
                    values, indices = torch.log_softmax(output, dim=1).topk(k)
                    for logp_c, c in zip(values[0], indices[0]):
                        if c !=0:
                            new_candidates.append((output, state, res, logp + logp_c, length+1, c))

            candidates = []
            for output, state, res, logp, length, c in sorted(new_candidates, key=lambda x: -x[3])[:k]:
                if c is None:
                    candidates.append((None, None, res, logp, length, True))
                else:
                    new_out, new_state = model.forward(c.view(1,1), state)
                    candidates.append((new_out[-1], new_state, res + [int(c)], logp, length, c == preprocessor.eos_index))

    return [c[2] for c in candidates]


display(HTML("<div><b>Avec RNN</b></div>"))
beams = generate_beam(preprocessor, rnn_model, 15, "i was happy")
print(preprocessor.decode(beams[0]))

display(HTML("<div><b>Avec LSTM</b></div>"))
beams = generate_beam(preprocessor, lstm_model, 15, "i was disappointed")
for txt in beams:
    print(preprocessor.decode(txt))



In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###